In [1]:
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, text

# Creazione del database
engine = create_engine('sqlite:///dati20.db')

# Funzione per creare le tabelle
def create_tables():
    with engine.connect() as conn:
        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS MOVIE (
            movie_id INTEGER PRIMARY KEY,
            nome_film TEXT,
            anno INTEGER
        )
        '''))

        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS SCRIPT (
            movie_id INTEGER PRIMARY KEY,
            script TEXT
        )
        '''))

        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS CAST (
            movie_id INTEGER,
            nome_attore TEXT,
            ruolo TEXT,
            PRIMARY KEY (movie_id, nome_attore)
        )
        '''))

create_tables()

# Funzione per scaricare lo script da IMSDb
def scarica_script_da_imsdb(titolo_film):
    url = f"https://www.imsdb.com/scripts/{titolo_film.replace(' ', '-')}.html"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        script = soup.find('td', {'class': 'scrtext'}).text
        return script
    else:
        print(f"Errore nel scaricare lo script da IMSDb per il film {titolo_film}")
        return None

# Funzione per scaricare il cast da IMDb
def scarica_cast_da_imdb(imdb_id):
    url = f"https://www.imdb.com/title/{imdb_id}/fullcredits"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        cast_list = []
        table = soup.find('table', class_='cast_list')
        if table:
            rows = table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) > 1:
                    actor_name = cols[1].get_text(strip=True)
                    role_name = cols[3].get_text(strip=True) if len(cols) > 3 else "Unknown"
                    cast_list.append({'name': actor_name, 'role': role_name})
        return cast_list
    else:
        print(f"Errore nel scaricare il cast da IMDb per il film {imdb_id}")
        return []

# Funzione per inserire i dati nel database
def insert_data(data, table_name):
    with engine.begin() as conn:
        if table_name == "SCRIPT":
            conn.execute(text('''
            INSERT INTO SCRIPT (movie_id, script)
            VALUES (:movie_id, :script)
            '''), data)
        elif table_name == "CAST":
            existing_entry = conn.execute(text('''
                SELECT 1
                FROM CAST
                WHERE movie_id = :movie_id AND nome_attore = :nome_attore
            '''), data).fetchone()
            if not existing_entry:
                conn.execute(text('''
                INSERT INTO CAST (movie_id, nome_attore, ruolo)
                VALUES (:movie_id, :nome_attore, :ruolo)
                '''), data)
        elif table_name == "MOVIE":
            conn.execute(text('''
            INSERT INTO MOVIE (movie_id, nome_film, anno)
            VALUES (:movie_id, :nome_film, :anno)
            '''), data)

# Funzione principale
def main():
    # Lista di film con titoli e ID IMDb
    films = [
        {"titolo": "Rocky", "imdb_id": "tt0075148", "anno": 1976},
        {"titolo": "Blade Runner","imdb_id": "tt0083658", "anno":1982 },
        {"titolo": "Dead Poets Society", "imdb_id": "tt0097165", "anno": 1989},
        {"titolo": "Fargo", "imdb_id": "tt0116282", "anno": 1996},
        {"titolo": "Good Will Hunting", "imdb_id": "tt0119217", "anno": 1997},
        {"titolo": "Truman Show, The", "imdb_id": "tt0120382", "anno": 1998},
        {"titolo": "American Beauty", "imdb_id": "tt0169547", "anno": 1999},
        {"titolo": "Gladiator", "imdb_id": "tt0172495", "anno": 2000},
        {"titolo": "Memento", "imdb_id": "tt0209144", "anno": 2000},
        {"titolo": "American Psycho", "imdb_id": "tt0144084", "anno": 2000},
        {"titolo": "Big Fish", "imdb_id": "tt0319061", "anno": 2003},
        {"titolo": "Eternal Sunshine of the Spotless Mind", "imdb_id": "tt0338013", "anno": 2004},
        {"titolo": "Devil Wears Prada, The", "imdb_id": "tt0458352", "anno": 2006},
        {"titolo": "No Country for Old Men", "imdb_id": "tt0477348", "anno": 2007},
        {"titolo": "Anna Karenina", "imdb_id": "tt1781769", "anno": 2012},
        {"titolo": "12 Years a Slave", "imdb_id": "tt2024544", "anno": 2013},
        {"titolo": "Great Gatsby, The", "imdb_id": "tt1343092", "anno": 2013},
        {"titolo": "Interstellar", "imdb_id": "tt0816692", "anno": 2014},
        {"titolo": "American Sniper", "imdb_id": "tt2179136", "anno": 2014},
        {"titolo": "La La Land", "imdb_id": "tt3783958", "anno": 2016},
    ]

    for movie_id, film in enumerate(films, start=1):
        titolo_film = film["titolo"]
        imdb_id = film["imdb_id"]
        anno = film["anno"]

        # Scarica il cast da IMDb
        cast_list = scarica_cast_da_imdb(imdb_id)

        # Inserisci il cast nel database
        for actor in cast_list:
            cast_data = {
                'movie_id': movie_id,
                'nome_attore': actor['name'],
                'ruolo': actor['role']
            }
            insert_data(cast_data, 'CAST')

        # Scarica lo script da IMSDb
        script = scarica_script_da_imsdb(titolo_film)
        if script:
            script_data = {
                'movie_id': movie_id,
                'script': script
            }
            insert_data(script_data, 'SCRIPT')

        # Inserisci i dati del film nella tabella MOVIE
        movie_data = {
            'movie_id': movie_id,
            'nome_film': titolo_film,
            'anno': anno
        }
        insert_data(movie_data, 'MOVIE')

if __name__ == "__main__":
    main()
